![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)

## Run SQL Server 2022 container image with Docker

This notebook will use Docker to pull and run the SQL Server 2022 container image and connect to it in Azure Data Studio

### Dependencies

- Docker Engine. For more information, see [Install Docker](https://docs.docker.com/engine/installation/).

<span style="color:red;"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### Check dependencies

In [1]:
import sys,os,getpass,json,html,time
from string import Template

def run_command(displayCommand = ""):
    print("Executing: " + displayCommand if displayCommand != "" else cmd)
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n')
    print(f'Command successfully executed')

cmd = 'docker version'
run_command()

docker version


Client:
 Cloud integration: v1.0.35+desktop.5
 Version:           24.0.6
 API version:       1.43
 Go version:        go1.20.7
 Git commit:        ed223bc
 Built:             Mon Sep  4 12:32:48 2023
 OS/Arch:           windows/amd64
 Context:           default

Server: Docker Desktop 4.25.2 (129061)
 Engine:
  Version:          24.0.6
  API version:      1.43 (minimum version 1.12)
  Go version:       go1.20.7
  Git commit:       1a79695
  Built:            Mon Sep  4 12:32:16 2023
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.6.22Command successfully executed
  GitCommit:        8165feabfdfe38c65b599c4993d227328c231fca
 runc:



  Version:          1.1.8
  GitCommit:        v1.1.8-0-g82f18fe
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


### List existing containers
You can view the ports that have been used by existing containers

In [2]:
cmd = f'docker ps -a'
run_command()

docker ps -a


CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Command successfully executed


### Required information

In [7]:
env_var_flag = "AZDATA_NB_VAR_DOCKER_PASSWORD" in os.environ
password_name = 'SQL Server sa account password'
if env_var_flag:
    sql_password = os.environ["AZDATA_NB_VAR_DOCKER_PASSWORD"]
    sql_port = os.environ["AZDATA_NB_VAR_DOCKER_PORT"]
else:
    sql_password = getpass.getpass(prompt = password_name)
    password_confirm = getpass.getpass(prompt = f'Confirm {password_name}')
    if sql_password != password_confirm:
        raise SystemExit(f'{password_name} does not match the confirmation password.')
    sql_port = input('SQL Server port, default value is 1433')
    if len(sql_port) == 0:
        sql_port = '1433'
print(f'{password_name}: ******')
print(f'Port: {sql_port}')

SQL Server sa account password: ******
Port: 1433


### Pull the container image

In [8]:
cmd = f'docker pull mcr.microsoft.com/mssql/server:2022-latest'
run_command()

docker pull mcr.microsoft.com/mssql/server:2022-latest


2022-latest: Pulling from mssql/server
Digest: sha256:ffef32dda16cc5abd70db1d0654c01cbe9f7093d66e0c83ade20738156cb7d0e
Status: Image is up to date for mcr.microsoft.com/mssql/server:2022-latest
mcr.microsoft.com/mssql/server:2022-latest
Command successfully executed



What's Next?
  View a summary of image vulnerabilities and recommendations â†’ docker scout quickview mcr.microsoft.com/mssql/server:2022-latest


### Start a new container

In [9]:
if env_var_flag:
    container_name = os.environ["AZDATA_NB_VAR_DOCKER_CONTAINER_NAME"]
else:
    container_name = 'sql2022-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
print('New container name: ' + container_name)

template = Template(f'docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD=$password" -p {sql_port}:1433 --name {container_name} -d mcr.microsoft.com/mssql/server:2022-latest')
cmd = template.substitute(password=sql_password)
run_command(template.substitute(password='******'))

New container name: SQL2022-Test
Executing: docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD=******" -p 1433:1433 --name SQL2022-Test -d mcr.microsoft.com/mssql/server:2022-latest


002b420b07e347e224a4be9d84ba80d5915f285dd69dc3998c5b4b8d05b225c0
Command successfully executed


### List all the containers

In [10]:
cmd = f'docker ps -a'
run_command()

docker ps -a


CONTAINER ID   IMAGE                                        COMMAND                  CREATED          STATUS          PORTS                    NAMES
002b420b07e3   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/permâ€¦"   13 seconds ago   Up 12 seconds   0.0.0.0:1433->1433/tcp   SQL2022-Test
Command successfully executed


In [17]:
from urllib import request

remote_url = 'https://github.com/phakkhaphong/MS-SQL-Query/blob/main/TSQL.bacpac'

file = 'TSQL.bacpac'

request.urlretrieve(remote_url, file)

('TSQL.bacpac', <http.client.HTTPMessage at 0x2d29572c3d0>)

In [21]:
cmd = f'docker cp TSQL.bacpac SQL2022-Test:/var/backups/TSQL.bacpac'
run_command()

docker cp TSQL.bacpac SQL2022-Test:/var/backups/TSQL.bacpac


Command successfully executed


### Connect to SQL Server in Azure Data Studio
It might take a couple minutes for SQL Server to launch

In [ ]:
from IPython.display import *
connectionParameter = '{"serverName":"localhost,' + sql_port + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(sql_password) + ',"options": {"trustServerCertificate":true}}'
display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to SQL Server</font></a><br/>'))
display(HTML('<br/><span style="color:red"><font size="2">NOTE: The SQL Server password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))

### Stop and remove the container

In [ ]:
stop_container_command = f'docker stop {container_name}'
remove_container_command = f'docker rm {container_name}'
display(HTML("Use this link to: <a href=\"command:workbench.action.terminal.focus\">open the terminal window in Azure Data Studio</a> and use the links below to paste the command to the terminal."))
display(HTML("Stop the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+stop_container_command.replace(" ","%20")+"%22%7D\">" + stop_container_command + "</a>"))
display(HTML("Remove the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+remove_container_command.replace(" ","%20")+"%22%7D\">" + remove_container_command + "</a>"))